In [10]:
import pandas as pd
import os
import sys
import torch
from pyrokinetics import Pyro, PyroScan
import numpy as np
from pathlib import Path

In [2]:
# helperfunctions path
sys.path.append("/home/Felix/Documents/Physics_Work/Project_Codes/UKAEAdigiLab_FullGyro/digiLabScripts/")
import helperfunctions as hf

# models path
models_path = "/home/Felix/Documents/Physics_Work/Project_Codes/UKAEAdigiLab_FullGyro/8d_loRes/"
models,name = hf.LoadModels(models_path)


M12  doesnt exist
M32  doesnt exist
Loading:  /home/Felix/Documents/Physics_Work/Project_Codes/UKAEAdigiLab_FullGyro/8d_loRes/output_growth_rate_log_warping_True_kernel_M52.pt
M12  doesnt exist
Loading:  /home/Felix/Documents/Physics_Work/Project_Codes/UKAEAdigiLab_FullGyro/8d_loRes/output_mode_frequency_log_warping_True_kernel_M32.pt
M52  doesnt exist
M12  doesnt exist
Loading:  /home/Felix/Documents/Physics_Work/Project_Codes/UKAEAdigiLab_FullGyro/8d_loRes/output_kperp2_phi_log_warping_True_kernel_M32.pt
M52  doesnt exist
Loading:  /home/Felix/Documents/Physics_Work/Project_Codes/UKAEAdigiLab_FullGyro/8d_loRes/output_kperp2_apa_log_warping_True_kernel_M12.pt
M32  doesnt exist
M52  doesnt exist
Loading:  /home/Felix/Documents/Physics_Work/Project_Codes/UKAEAdigiLab_FullGyro/8d_loRes/output_kperp2_bpar_log_warping_True_kernel_M12.pt
M32  doesnt exist
M52  doesnt exist
M12  doesnt exist
M32  doesnt exist
Loading:  /home/Felix/Documents/Physics_Work/Project_Codes/UKAEAdigiLab_FullGyro/8d

In [15]:
Template_Path = "/home/Felix/Documents/Physics_Work/Project_Codes/GP_Model_Eval/GS2/Templates/r1.in"
project_dir = Path("/home/Felix/Documents/Physics_Work/Project_Codes/GP_Model_Eval")


In [16]:
def Read_from_gs2(step_run):

    in_loc = project_dir / "GS2" / "Templates" / step_run / "gs2.in"

    pyro = Pyro(gk_file=in_loc, gk_code="GS2")


    pyro.gk_code = "TGLF"

    pyro.numerics.nonlinear = True

    # Use existing parameter with more realistic ky range
    param_1 = "ky" 
    values_1 = np.arange(0.1, 1, 0.1)/pyro.norms.pyrokinetics.rhoref
    
    # Add beta parameter with realistic values
    param_2 = "gamma_exb"
    values_2 = np.arange(0, 0.2, 0.1)*pyro.norms.pyrokinetics.vref/pyro.norms.pyrokinetics.lref
  
    # Dictionary of param and values
    param_dict = {param_1: values_1, param_2: values_2}
  

    # Create PyroScan object with more descriptive naming
    pyro_scan = PyroScan(
        pyro,
        param_dict,
        value_fmt=".4f",  # Increased precision for small beta values
        value_separator="_",
        parameter_separator="_",
    )

    # Add proper parameter mapping for beta
    pyro_scan.add_parameter_key(
        parameter_key="gamma_exb",
        parameter_attr="numerics", 
        parameter_location=["gamma_exb"]
    )

    # Create scan directory and write input files
    try:
        pyro_scan.write(
            file_name="input.tglf",
            base_directory=project_dir / "TGLF_Runs" / step_run ,
            template_file=None
        )
    except Exception as e:
        print(f"Error writing parameter scan files: {e}")
        return None

    return pyro_scan

/home/Felix/Documents/Physics_Work/Project_Codes/GP_Model_Eval/pyrokinetics/src/pyrokinetics/pyro.py:1651: UserWarning: Explicitly set value of beta (0.09092868468130633 beta_ref_ee_B0) is inconsistent with value from physical reference values (0.09609377154804474 beta_ref_ee_B0)
  warnings.warn(
/home/Felix/Documents/Physics_Work/Project_Codes/GP_Model_Eval/pyrokinetics/src/pyrokinetics/pyro.py:345: UserWarning: Setting gk_code to TGLF without first reading a gyrokinetics file of that type. Creating a new context using the default template file for TGLF and data from the previous context (if available).
  warnings.warn(warn_msg, UserWarning)
